#### Импорт

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt

#### Загрузка набора инструментов

In [2]:
try:
    sys.path.insert(1, '../../../..');
    from assets.dataloader.parse_lvds import Parser
except:
    print('Не удалось загрузить модуль с компьютера.');
    ans = str(input('Скачать модуль из репозитория ? '));
    if ans == 'Да' or 'Д' or 'д' or 'Yes' or 'Y' or 'y':
        !git clone https://github.com/imaks79/assets.git
        from assets.dataloader.parse_lvds import Parser
    else: print('Не удалось загрузить модуль');

#### ```Чтение``` необработанных данных

In [4]:
parser = Parser('data');
print('Файлов доступно для чтения:', parser.files);

Файлов доступно для чтения: ['adc_data.bin']


#### ```Организация``` необработанных данных

In [5]:
adcdata = parser.organize(file_idx = 0);
print(f'(numFrames, numChirps, numChannels, numADCSamples) = {adcdata.shape}');

(numFrames, numChirps, numChannels, numADCSamples) = (1024, 128, 4, 256)


#### ```Создание``` набора

In [6]:
chirp = adcdata[0, 0, 0, :];
simple_frame = adcdata[30, :, 0, :];
MIMO_frame = adcdata[30, :, :, :];

#### Сохранение данных для ```анализа```

In [7]:
np.save('data/simple_chirp.npy', chirp);
np.save('data/simple_frame.npy', simple_frame);
np.save('data/MIMO_frame.npy', MIMO_frame);

#### SIMPLE CHIRP

In [92]:
range_profile = np.fft.fft(chirp);

#### SIMPLE FRAME

In [ ]:
range_plot = np.fft.fft(simple_frame, axis = 1);

In [93]:
range_doppler = np.fft.fft(simple_frame, axis = 0);

#### MIMO FRAME

In [94]:
range_plot = np.fft.fft(MIMO_frame, axis = 1);
range_doppler = np.fft.fft(range_plot, axis = 0);

num_angle_bins = 64;
padding = ((0, 0), (0, num_angle_bins - range_doppler.shape[1]), (0, 0));
range_azimuth = np.pad(range_doppler, padding, mode = 'constant');
range_azimuth = np.fft.fft(range_azimuth, axis = 1);
range_azimuth = np.log(np.abs(range_azimuth).sum(0).T);